In [1]:
import numpy as np
import pandas as pd
import keras

from keras import Input, layers
from keras.utils import plot_model

# Helper classes for data processing and state representation
from reinforcetrader.data_pipeline import RawDataLoader, FeatureBuilder
from reinforcetrader.state import EpisodeStateLoader
from reinforcetrader.dqn_agent import RLAgent

# Helper method for displaying large dataframes
from IPython.display import HTML
pd.set_option("display.max_rows", None)
from IPython.display import HTML, display

def display_df(df, rows=10):
    # Puts the scrollbar next to the DataFrame
    display(HTML("<div style='height: 300px; overflow: auto; width: 98%'>" + df.head(rows).to_html() + "</div>"))

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


ModuleNotFoundError: No module named 'wrapt'

# Data Pre-processing and Feature Building

In [2]:
# Get DJI raw data using Yahoo Finance API
data_loader = RawDataLoader(start_date='2000-02-20', end_date='2025-08-11')

# Download or load from cache the data
raw_data = data_loader.get_hist_prices()

# Inspect the raw DJI close and volume data
display_df(raw_data)

Loading cached data from data\raw\DJI_tickers_data_2000-02-20_2025-08-11.csv


Ticker,AAPL,AMGN,AMZN,AXP,BA,CAT,CSCO,CVX,DIS,GS,HD,HON,IBM,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,NVDA,PG,SHW,TRV,UNH,VZ,WMT,AAPL,AMGN,AMZN,AXP,BA,CAT,CSCO,CVX,DIS,GS,HD,HON,IBM,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,NVDA,PG,SHW,TRV,UNH,VZ,WMT,AAPL,AMGN,AMZN,AXP,BA,CAT,CSCO,CVX,DIS,GS,HD,HON,IBM,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,NVDA,PG,SHW,TRV,UNH,VZ,WMT,AAPL,AMGN,AMZN,AXP,BA,CAT,CSCO,CVX,DIS,GS,HD,HON,IBM,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,NVDA,PG,SHW,TRV,UNH,VZ,WMT,AAPL,AMGN,AMZN,AXP,BA,CAT,CSCO,CVX,DIS,GS,HD,HON,IBM,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,NVDA,PG,SHW,TRV,UNH,VZ,WMT
Price,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,Close,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Open,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-02-22,0.854166,46.676090,3.178125,28.713049,25.179064,9.998342,40.554924,14.651802,25.399883,57.333981,31.572769,25.384874,55.299919,20.309631,24.652752,12.682282,17.359310,18.637793,24.871798,28.753778,2.537750,0.125123,23.644180,4.486305,12.680239,5.775242,13.901404,10.256790,0.877620,49.744611,3.325000,28.945231,25.867238,10.145136,41.577489,14.748593,26.969539,57.905124,32.713510,25.384874,56.514276,20.422642,25.171337,12.896717,17.829388,18.702416,24.871798,29.769068,2.702234,0.126317,23.787382,4.514521,13.012182,5.855027,14.051061,10.509879,0.800693,45.898155,3.153125,28.403475,23.235985,9.786305,39.675518,14.373527,24.829099,56.718903,31.278384,23.101578,54.178977,19.825300,23.974601,12.375947,17.292156,17.888145,23.471572,28.447276,2.537750,0.120348,22.928172,4.443982,12.248712,5.664770,13.618721,9.963738,0.826491,49.571737,3.246875,28.790443,23.316947,9.835236,41.413879,14.530813,26.969539,57.114310,31.867154,23.672403,55.798117,20.148188,24.253841,12.590382,17.325733,18.146644,23.732632,29.156063,2.690485,0.124647,23.500979,4.500413,12.547461,5.695456,13.967918,10.496559,422296000,8657100,113716000,13781561,4771400,4028800,61144400,3526400,12744146,720100,8019200,2590601,8006921,7307400,7068600,10543000,4479500,3449982,9640971,66296200,12847200,542496000,5226800,1626000,966400,5828800,3589701,37027800
2000-02-23,0.872459,49.398842,3.521875,28.506676,24.045612,9.867865,45.361000,14.179937,25.970661,60.277523,32.271908,24.276808,54.179008,19.663868,24.692656,12.559752,17.527193,19.213808,23.969963,28.887878,2.637616,0.125840,23.071375,4.514521,12.613860,5.621809,13.369297,10.017026,0.893099,49.874246,3.587500,29.035534,25.057633,10.096212,45.483708,14.530806,26.351182,61.156204,32.382302,25.384880,55.050852,20.228922,25.271079,12.605703,17.930117,19.552034,25.061664,29.347632,2.678736,0.127154,23.644181,4.542736,12.813026,5.744555,13.851523,10.336719,0.833058,46.416760,3.234375,28.210000,24.005132,9.704759,41.332092,14.095245,24.829092,56.982469,31.131168,23.605250,54.054459,19.663868,24.433364,12.253417,17.392885,18.901600,23.969963,28.217404,2.625867,0.122139,22.944085,4.373441,12.481083,5.584984,13.319412,9.963743,0.849827,47.497224,3.243750,28.687263,25.057633,9.998349,41.557057,14.518707,25.399877,57.158205,31.903927,25.384880,55.050852,20.180488,24.692656,12.605703,17.426462,19.018679,25.061664,28.658001,2.655239,0.123691,23.644181,4.542736,12.613860,5.720006,13.602096,10.323397,473099200,9098600,168824000,10902205,3880800,4894400,105707800,3770000,8371718,702900

In [3]:
# Instantiate Feature Builder to Build relevant features for state space
feature_builder = FeatureBuilder(hist_prices=raw_data)

# Build the features
feature_builder.build_features()
features_data = feature_builder.get_features()

# Inspect the features
display_df(features_data, rows=5)

Building ticker features: 100%|█████████████████████████████████████| 28/28 [00:03<00:00,  9.02it/s]


File already exists, skipping save: data\processed\DJI_tickers_features_2000-03-28_2025-08-08.csv


# State Representation

In [4]:
esl = EpisodeStateLoader(features_data, 'configs/episodes.json')

In [5]:
esl.get_state_matrix('train', 1, 'AAPL', 2, 4)

array([[ 0.38463455,  0.5897253 ,  0.02564011,  0.00802265,  0.0434343 ,
         0.04222209,  0.82911706,  0.26658493,  0.28503725,  0.        ,
         0.5313614 ],
       [ 0.38463455,  0.5897253 ,  0.02564011,  0.00802265,  0.0434343 ,
         0.04222209,  0.82911706,  0.26658493,  0.28503725,  0.        ,
         0.5313614 ],
       [-0.6128125 ,  0.01113928,  0.37604818, -0.01009882,  0.03409885,
         0.04086558,  0.723041  ,  0.27118123,  0.1979622 ,  0.        ,
         0.70516175],
       [-0.6377509 ,  0.3367397 ,  0.02550943, -0.05781399,  0.01272239,
         0.034076  ,  0.42373735,  0.2678086 , -0.02859687,  0.34759662,
         1.2002095 ]], dtype=float32)

In [6]:
esl.get_state_OHLCV('train', 1, 'AAPL', 1)

array([1.0460138e+00, 1.0464828e+00, 1.0043840e+00, 1.0202152e+00,
       2.3983680e+08], dtype=float32)

# Deep Q-Network and RL Agent

In [7]:
# Init the RL agent and DQN Model
window_size = 26
agent = RLAgent(window_size=window_size, num_features=(3, 8))

In [8]:
agent.train(esl, [1], 64, 5)

KeyboardInterrupt: 

In [9]:
import tensorflow as tf
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("Physical GPUs:", tf.config.list_physical_devices("GPU"))

Built with CUDA: False
Physical GPUs: []
